In [83]:
import pandas as pd

In [84]:
df = pd.read_csv("data/restaurant_scores.csv")
df.columns

Index(['business_id', 'business_name', 'business_address',
       'business_postal_code', 'business_latitude', 'business_longitude',
       'business_location', 'business_phone_number', 'inspection_id',
       'inspection_date', 'inspection_score', 'violation_id',
       'violation_description', 'risk_category'],
      dtype='object')

In [85]:
df['date'] = pd.to_datetime(df['inspection_date'])

In [86]:
print(df.inspection_score.min())
print(df.inspection_score.max())

52
100


In [87]:
print(df.date.min())
print(df.date.max())

2015-10-02 00:00:00
2018-10-01 00:00:00


In [111]:
df.violation_description.unique()

array(['Food safety certificate or food handler card not available',
       'Unclean nonfood contact surfaces', nan,
       'Unclean or degraded floors walls or ceilings',
       'Moderate risk vermin infestation',
       'Inadequate and inaccessible handwashing facilities',
       'Unapproved or unmaintained equipment or utensils',
       'Foods not protected from contamination',
       'High risk food holding temperature', 'Improper thawing methods',
       'Improper storage of equipment utensils or linens',
       'Moderate risk food holding temperature',
       'No thermometers or uncalibrated thermometers',
       'High risk vermin infestation', 'Improper food storage',
       'Improper storage use or identification of toxic substances',
       'Improper or defective plumbing',
       'Inadequately cleaned or sanitized food contact surfaces',
       'Permit license or inspection report not posted',
       'Inadequate ventilation or lighting',
       'Wiping cloths not clean or pro

In [88]:
idx = df.groupby('business_name')['date'].transform(max) == df['date']
df_latest = df[idx]
print(len(df))
print(len(df_latest))

5796
1993


In [89]:
df_latest = df_latest.loc[pd.notnull(df_latest.violation_description)]
print(len(df_latest))
print(len(df_latest.business_name.unique()))

1891
613


In [90]:
risks = df_latest.groupby('business_name')['violation_description'].agg(lambda x: ', '.join(x))
df_risks = pd.DataFrame({'business_name':risks.index, 'risks':risks.values})

In [110]:
df_risks = pd.DataFrame({'business_name':risks.index, 'risks':risks.values})
df_new = pd.merge(df_latest, df_risks, how='inner', on=['business_name'])
df_new.head(1)

,business_id,business_name,business_address,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,inspection_id,inspection_date,inspection_score,violation_id,violation_description,risk_category,date,risks
0,69769,Whole Foods Market,1150 Ocean Ave,94112,37.723652,-122.45461,"(37.723652, -122.45461)",14155337400,69769_20180725,7/25/18,90,69769_20180725_103144,Unapproved or unmaintained equipment or utensils,Low Risk,2018-07-25,Unapproved or unmaintained equipment or utensi...


In [105]:
print(len(df_new))
df_new.drop(columns=['business_location', 'date',
                     'inspection_id', 'violation_id', 'violation_description', 
                     'risk_category'], inplace=True)
df_new.drop_duplicates(keep='first', inplace=True)
print(len(df_new))

1891
613


In [108]:
df_new.to_csv('data/restaurant_scores_transformed.csv')